In [2]:
import numpy as np
import torch
import math

## Computer quantile for kl_loss_corner

In [3]:
def compute_score(data_path):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    scores = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        score = np.abs(pred-target) / np.sqrt(np.exp(cov))
        for s in score:
            scores.append(list(s))
    return np.array(scores)

def compute_quantile(scores, alpha):
    n = scores.shape[0]
    q_level = np.ceil((n+1)*(1-alpha))/n
    quantile = []
    for i in range(scores.shape[1]):
        q = np.quantile(scores[:,i], q_level, interpolation='higher')
        quantile.append(q)
    return quantile

### disco

In [4]:
data_path = "check/check_loss_two_step_corner_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[5.685563231556715, 5.7333907000666064, 5.8156107496472655, 5.698934531905109, 5.349435233304022, 5.286866601436044, 5.364550922522339, 5.254597136256393]


### upperbound

In [5]:
data_path = "check/check_loss_two_step_corner_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[4.190345185940821, 4.351993435985297, 4.558479310459919, 4.717549387208282, 4.278715029642637, 4.102446480783583, 4.1725601258012475, 3.530299361843318]


### lowerbound

In [6]:
data_path = "check/check_loss_two_step_corner_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[6.10302951960435, 6.182331226887726, 5.740435199991636, 6.236356538776135, 6.334263986132204, 5.1828751359733305, 6.399557006521807, 5.073939114533967]
